In [1]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [2]:
# Database setup
conn = sqlite3.connect('bmi_data.db')
c = conn.cursor()
c.execute('''
          CREATE TABLE IF NOT EXISTS users (
              id INTEGER PRIMARY KEY AUTOINCREMENT,
              name TEXT,
              weight REAL,
              height REAL,
              bmi REAL,
              timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
          )
          ''')
conn.commit()

In [3]:
conn

In [4]:

class BMIApp:
    def __init__(self, root):
        self.root = root
        self.root.title("BMI Calculator")

        self.name_label = tk.Label(root, text="Name:")
        self.name_label.grid(row=0, column=0, padx=10, pady=10)

        self.name_entry = tk.Entry(root)
        self.name_entry.grid(row=0, column=1, padx=10, pady=10)

        self.weight_label = tk.Label(root, text="Weight (kg):")
        self.weight_label.grid(row=1, column=0, padx=10, pady=10)

        self.weight_entry = tk.Entry(root)
        self.weight_entry.grid(row=1, column=1, padx=10, pady=10)

        self.height_label = tk.Label(root, text="Height (m):")
        self.height_label.grid(row=2, column=0, padx=10, pady=10)

        self.height_entry = tk.Entry(root)
        self.height_entry.grid(row=2, column=1, padx=10, pady=10)

        self.calculate_button = tk.Button(root, text="Calculate BMI", command=self.calculate_bmi)
        self.calculate_button.grid(row=3, column=0, columnspan=2, pady=10)

        self.view_history_button = tk.Button(root, text="View History", command=self.view_history)
        self.view_history_button.grid(row=4, column=0, columnspan=2, pady=10)

    def calculate_bmi(self):
        try:
            weight = float(self.weight_entry.get())
            height = float(self.height_entry.get())
            bmi = round(weight / (height ** 2), 3)*100

            name = self.name_entry.get()

            # Save data to the database
            c.execute('''
                      INSERT INTO users (name, weight, height, bmi)
                      VALUES (?, ?, ?, ?)
                      ''', (name, weight, height, bmi))
            conn.commit()

            if bmi>24.9:
                messagebox.showinfo("BMI Result", f"Your BMI is: {bmi} \n which is considered Overweight.")
            elif bmi<18.4:
                messagebox.showinfo("BMI Result", f"Your BMI is: {bmi} \n which is considered Underweight.")
            else:
                messagebox.showinfo("BMI Result", f"Your BMI is: {bmi} \n which is considered Normal.")

            
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numerical values for weight and height.")

    def view_history(self):
        # Fetch data from the database
        c.execute('''
                  SELECT name, bmi, timestamp
                  FROM users
                  ORDER BY timestamp DESC
                  ''')
        data = c.fetchall()

        if data:
            names, bmis, timestamps = zip(*data)

            # Plot BMI trend
            plt.figure(figsize=(8, 4))
            plt.plot(timestamps, bmis, marker='o')
            plt.title('BMI Trend Over Time')
            plt.xlabel('Timestamp')
            plt.ylabel('BMI')
            plt.xticks(rotation=45)

            # Show the plot in the GUI
            self.plot_window = tk.Toplevel(self.root)
            self.plot_window.title("BMI Trend")
            self.plot_frame = tk.Frame(self.plot_window)
            self.plot_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=1)
            self.canvas = FigureCanvasTkAgg(plt.gcf(), master=self.plot_frame)
            self.canvas.draw()
            self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        else:
            messagebox.showinfo("No Data", "No BMI data available.")

if __name__ == "__main__":
    root = tk.Tk()
    app = BMIApp(root)
    root.mainloop()

# Close the database connection after the GUI is closed
conn.close()
